In [1]:
import sys
sys.path.insert(0, '..')

In [2]:
import pandas as pd

df_ground_truth = pd.read_csv('./ground_truth_evidently.csv')

In [3]:
df_sample = df_ground_truth.sample(25)

In [4]:
ground_truth = df_sample.to_dict(orient='records')

In [5]:
q = ground_truth[10]

In [6]:
q

{'question': 'create Dataset object Evidently',
 'summary_answer': 'To create a `Dataset` object in Evidently, you can use the `Dataset.from_pandas()` method with a data definition to specify the roles and types of your data columns.',
 'difficulty': 'beginner',
 'intent': 'code',
 'filename': 'docs/library/data_definition.mdx'}

In [7]:
import main
agent = main.agent

In [8]:
import asyncio
from tqdm.auto import tqdm

async def map_progress(seq, f, max_concurrency=6):
    """Asynchronously map async function f over seq with progress bar."""
    semaphore = asyncio.Semaphore(max_concurrency)

    async def run(el):
        async with semaphore:
            return await f(el)

    # create one coroutine per element
    coros = [run(el) for el in seq]

    # turn them into tasks that complete as they finish
    completed = asyncio.as_completed(coros)

    results = []

    for coro in tqdm(completed, total=len(seq)):
        result = await coro
        results.append(result)

    return results


In [10]:
import traceback

async def run_agent(q):
    try:
        result = await agent.run(q['question'])
        return (q, result)
    except:
        print(f'error processing {q}')
        traceback.print_exc()
        return (None, None)


In [17]:
all_results = await map_progress(ground_truth, run_agent, max_concurrency=10)

  0%|          | 0/25 [00:00<?, ?it/s]

forcing output
forcing output
forcing output
forcing output
error processing {'question': 'Evidently mapping roles and types', 'summary_answer': 'The article discusses how to accurately specify roles and types for each column in your dataset to avoid errors in evaluations.', 'difficulty': 'advanced', 'intent': 'text', 'filename': 'docs/library/data_definition.mdx'}


Traceback (most recent call last):
  File "/Users/yenchunchen/Desktop/Project/ai-bootcamp/week3/code/.venv/lib/python3.12/site-packages/pydantic_ai/models/openai.py", line 493, in _completions_create
    return await self.client.chat.completions.create(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yenchunchen/Desktop/Project/ai-bootcamp/week3/code/.venv/lib/python3.12/site-packages/openai/resources/chat/completions/completions.py", line 2603, in create
    return await self._post(
           ^^^^^^^^^^^^^^^^^
  File "/Users/yenchunchen/Desktop/Project/ai-bootcamp/week3/code/.venv/lib/python3.12/site-packages/openai/_base_client.py", line 1794, in post
    return await self.request(cast_to, opts, stream=stream, stream_cls=stream_cls)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yenchunchen/Desktop/Project/ai-bootcamp/week3/code/.venv/lib/python3.12/site-packages/openai/_base_client.py", line 1594, in request
  

error processing {'question': 'create Dataset object Evidently', 'summary_answer': 'To create a `Dataset` object in Evidently, you can use the `Dataset.from_pandas()` method with a data definition to specify the roles and types of your data columns.', 'difficulty': 'beginner', 'intent': 'code', 'filename': 'docs/library/data_definition.mdx'}


Traceback (most recent call last):
  File "/Users/yenchunchen/Desktop/Project/ai-bootcamp/week3/code/.venv/lib/python3.12/site-packages/pydantic_ai/models/openai.py", line 493, in _completions_create
    return await self.client.chat.completions.create(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yenchunchen/Desktop/Project/ai-bootcamp/week3/code/.venv/lib/python3.12/site-packages/openai/resources/chat/completions/completions.py", line 2603, in create
    return await self._post(
           ^^^^^^^^^^^^^^^^^
  File "/Users/yenchunchen/Desktop/Project/ai-bootcamp/week3/code/.venv/lib/python3.12/site-packages/openai/_base_client.py", line 1794, in post
    return await self.request(cast_to, opts, stream=stream, stream_cls=stream_cls)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yenchunchen/Desktop/Project/ai-bootcamp/week3/code/.venv/lib/python3.12/site-packages/openai/_base_client.py", line 1594, in request
  

In [51]:
import pickle 

with open('all-results-v1.bin', 'wb') as f_out:
    pickle.dump(all_results, f_out)

In [18]:
q, r = all_results[10]

In [19]:
q

{'question': 'install Evidently for LLM',
 'summary_answer': 'Instructions for installing the Evidently library to use for creating and evaluating LLM judges are provided within the tutorial.',
 'difficulty': 'beginner',
 'intent': 'code',
 'filename': 'examples/LLM_judge.mdx'}

In [20]:
r.usage()

RunUsage(input_tokens=12459, output_tokens=465, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, requests=2, tool_calls=5)

In [21]:
from toyaikit.pricing import PricingConfig
pricing = PricingConfig()


def calculate_cost(model, all_results):
    total_input = 0
    total_output = 0
    
    for q, result in all_results:
        if result is None:
            continue
        usage = result.usage()
        total_input = total_input + usage.input_tokens
        total_output = total_output + usage.output_tokens
    
    return pricing.calculate_cost(model, total_input, total_output)

In [22]:
calculate_cost('gpt-4o-mini', all_results)

CostInfo(input_cost=0.04618365, output_cost=0.0082908, total_cost=0.05447445)

In [23]:
import json

def simplify_messages(messages):
    messages_simplified = []

    for m in messages:
        parts = []

        for original_part in m.parts:
            kind = original_part.part_kind
            # print(original_part)
            part = {
                'kind': kind
            }
            if kind == 'user-prompt':
                part['content'] = original_part.content
            if kind == 'tool-call':
                if original_part.tool_name == 'final_result':
                    continue
    
                part['tool_name'] = original_part.tool_name
                part['args'] = json.loads(original_part.args)
            if kind == 'tool-return':
                continue
            if kind == 'text':
                part['content'] = original_part.content

            parts.append(part)

        if len(parts) > 0:
            messages_simplified.extend(parts)

    return messages_simplified

def count_tool_calls(messages):
    cnt = 0 
    for m in messages:
        if m['kind'] == 'tool-call':
            cnt = cnt + 1
    return cnt



In [24]:
# building and inspecting evaluation results
# log all the data


def process_result(q, result):
    row = {}

    row['question'] = q['question']
    row['answer'] = result.output.format_article()
    row['messages'] = simplify_messages(result.new_messages())
    row['num_tool_calls'] = count_tool_calls(row['messages']) 

    row['original_question'] = q
    row['original_result'] = result

    return row


rows = []

for q, r in all_results:
    if r is None:
        continue
    
    usage = r.usage()
    row = process_result(q, r)
    # row = {
    #     'question': q['question'],
    #     'answer': r.output.format_article(),
    #     'tool_call_number': usage.tool_calls,
    #     'requests': usage.requests
    # }
    rows.append(row)


In [25]:
rows[0]

{'question': 'SemanticSimilarity descriptor explained',
 'answer': "# Understanding SemanticSimilarity Descriptor in Evidently\n\n## What is the SemanticSimilarity Descriptor?\n\nThe `SemanticSimilarity` descriptor in Evidently is designed to evaluate the similarity between pairs of text data. It leverages models from the HuggingFace library for tasks related to natural language understanding. This can include evaluating aspects such as semantic relationships, sentiment similarity, or contextual coherence depending on the model used. The `SemanticSimilarity` descriptor essentially helps in quantifying how close the meanings of two texts are, providing valuable insights especially in applications like chatbots, content recommendations, or any system where understanding text relationships is crucial.\n\n### References\n- [Use HuggingFace models](https://github.com/evidentlyai/docs/blob/main/metrics/customize_hf_descriptor.mdx)\n## How to Use SemanticSimilarity?\n\nTo use the `SemanticSim

In [27]:
df_run = pd.DataFrame(rows)

df_run.head()

,question,answer,messages,num_tool_calls,original_question,original_result
0,SemanticSimilarity descriptor explained,# Understanding SemanticSimilarity Descriptor ...,"[{'kind': 'user-prompt', 'content': 'SemanticS...",5,{'question': 'SemanticSimilarity descriptor ex...,AgentRunResult(output=SearchResultArticle(foun...
1,using is_critical parameter in alerts,# Using `is_critical` Parameter in Evidently A...,"[{'kind': 'user-prompt', 'content': 'using is_...",4,{'question': 'using is_critical parameter in a...,AgentRunResult(output=SearchResultArticle(foun...
2,Quickstart Evidently setup,# Quickstart Setup for Evidently\n\n## Creatin...,"[{'kind': 'user-prompt', 'content': 'Quickstar...",5,"{'question': 'Quickstart Evidently setup', 'su...",AgentRunResult(output=SearchResultArticle(foun...
3,Drift detection for numerical columns,# Drift Detection for Numerical Columns in Evi...,"[{'kind': 'user-prompt', 'content': 'Drift det...",5,{'question': 'Drift detection for numerical co...,AgentRunResult(output=SearchResultArticle(foun...
4,non-letter character percentage function,# Non-letter Character Percentage Function in ...,"[{'kind': 'user-prompt', 'content': 'non-lette...",5,{'question': 'non-letter character percentage ...,AgentRunResult(output=SearchResultArticle(foun...


In [28]:
import pickle

In [32]:
with open('eval-run-v2-2025-10-29-20-15.bin', 'wb') as f_out:
    pickle.dump(rows, f_out)

In [31]:
!ls -lh

total 2408
-rw-r--r--@ 1 yenchunchen  staff   386K Oct 28 21:48 all-results-v1.bin
-rw-r--r--@ 1 yenchunchen  staff   433K Oct 29 20:15 eval-run-v2-2025-10-29-20-15.bin
-rw-r--r--  1 yenchunchen  staff    26K Oct 29 08:12 evaluation-agent-run.ipynb
-rw-r--r--  1 yenchunchen  staff   226K Oct 29 08:12 ground-truth.ipynb
-rw-r--r--@ 1 yenchunchen  staff   104K Oct 24 18:43 ground_truth_evidently.csv
-rw-r--r--  1 yenchunchen  staff    13K Oct 29 08:12 retrieval-evaluation.ipynb
